In [72]:
import numpy as np
import random
import sympy as sp
from numpy.linalg import matrix_rank
np.set_printoptions(threshold=np.inf)


# Dictionaries
char_dict = {'f' : [0,0,0,0],
             'g' : [0,0,0,1],
             'h' : [0,0,1,0],
             'i' : [0,0,1,1],
             'j' : [0,1,0,0],
             'k' : [0,1,0,1],
             'l' : [0,1,1,0],
             'm' : [0,1,1,1],
             'n' : [1,0,0,0],
             'o' : [1,0,0,1],
             'p' : [1,0,1,0],
             'q' : [1,0,1,1],
             'r' : [1,1,0,0],
             's' : [1,1,0,1],
             't' : [1,1,1,0],
             'u' : [1,1,1,1]}

inv_dict = {'0000': 'f',
            '0001': 'g',
            '0010': 'h',
            '0011': 'i',
            '0100': 'j',
            '0101': 'k',
            '0110': 'l',
            '0111': 'm',
            '1000': 'n',
            '1001': 'o',
            '1010': 'p',
            '1011': 'q',
            '1100': 'r',
            '1101': 's',
            '1110': 't',
            '1111': 'u'}

# It converts byte to corresponding two characters
def byte_str(byte):
    bin_num = '{:0>8}'.format(format(byte,"b"))
    char1 = inv_dict[bin_num[0:4]]
    char2 = inv_dict[bin_num[4:8]]
    return char1 + char2

# It maps set of two characters (byte) to its corresponding hex value
def map_to_str(byte_set):
    char_val = chr(16*(ord(byte_set[0]) - ord('f')) + ord(byte_set[1]) - ord('f'))
    return char_val

# It takes a full block of input eg 'fffffffffffffffg' and outputs corresponding hex list
def block_to_byte(input_block):
    plain_text = ""
    for i in range(0, len(input_block), 2):
        plain_text += map_to_str(input_block[i:i+2])
    return plain_text


In [73]:
from pyfinite import ffield

#Initialize finite field with characteristic 2 and degree 7
F = ffield.FField(7)

#Exponentiation table to store previously computed values for faster lookup
exp_table = [[-1]*128 for i in range(128)]

#Add two elements in the field
def add(a, b):
    return int(a) ^ int(b)

#Multiply two elements in the field
def multiply(a, b):
    return F.Multiply(a, b)

#Compute a^n in the field using exponentiation by squaring
def power(a, n):
    if exp_table[a][n] != -1:
        return exp_table[a][n]
    
    result = 0
    if n == 0:
        result = 1
    elif n == 1:
        result = a
    elif n % 2 == 0:
        sqrt_a = power(a, n // 2)
        result = multiply(sqrt_a, sqrt_a)
    else:
        sqrt_a = power(a, n // 2)
        result = multiply(multiply(sqrt_a, sqrt_a), a)
    
    exp_table[a][n] = result
    return result

#Add two vectors in the field
def add_vectors(v1, v2):
    result = [0]*8
    for i, (e1, e2) in enumerate(zip(v1, v2)):
        result[i] = add(e1, e2)
    return result

#Multiply a vector with an element in the field
def scalar_multiply(v, elem):
    result = [0]*8
    for i, e in enumerate(v):
        result[i] = multiply(e, elem)
    return result

#Apply a linear transformation matrix to a vector in the field
def linear_transform(mat, v):
    result = [0]*8
    for row, elem in zip(mat, v):
        result = add_vectors(scalar_multiply(row, elem), result)
    return result


In [74]:
# Initialize lists to store all possible exponents and diagonal values
poss_exp = [[] for i in range(8)]
poss_aii = [[[] for i in range(8)] for j in range(8)]

# Open the input and output files
with open("inputs.txt", 'r') as input_file, open("outputs.txt", 'r') as output_file:
    # Iterate over the lines in the input and output files
    for ind, (iline, oline) in enumerate(zip(input_file.readlines(), output_file.readlines())):
        inpString = []
        outString = []
        # Convert each input and output hex string to corresponding byte values
        for hexi in iline.strip().split(" "):
            inpString.append(block_to_byte(hexi)[ind])
        for hexo in oline.strip().split(" "):
            outString.append(block_to_byte(hexo)[ind])
            
        # Iterate over all possible values of exponents and diagonal values
        for i in range(1, 127):
            for j in range(1, 128):
                flag = True
                for inps, outs in zip(inpString, outString):
                    # Check if the input maps to the output for the current exponent and diagonal value
                    if ord(outs) != Expo(Multiply(Expo(Multiply(Expo(ord(inps), i), j), i), j), i):
                        flag = False
                        break
                if flag:
                    # If the input maps to the output for the current exponent and diagonal value,
                    # append them to the corresponding lists
                    poss_exp[ind].append(i)
                    poss_aii[ind][ind].append(j)

# Print the list of all possible diagonal values and exponents
print(poss_aii)
print(poss_exp)


[[[84, 28, 113], [], [], [], [], [], [], []], [[], [72, 101, 70], [], [], [], [], [], []], [[], [], [105, 43, 107], [], [], [], [], []], [[], [], [], [31, 11, 12], [], [], [], []], [[], [], [], [], [5, 31, 112], [], [], []], [[], [], [], [], [], [11, 41, 127], [], []], [[], [], [], [], [], [], [27, 20, 124], []], [[], [], [], [], [], [], [], [38, 61, 125]]]
[[18, 21, 88], [53, 83, 118], [17, 41, 69], [9, 44, 74], [78, 85, 91], [53, 83, 118], [26, 113, 115], [17, 41, 69]]


In [75]:
# Open input and output files
with open("inputs.txt", 'r') as input_file, open("outputs.txt", 'r') as output_file:
    
    # Iterate over each line in both input and output files
    for ind, (iline, oline) in enumerate(zip(input_file.readlines(), output_file.readlines())):
        
        # Consider only the first 6 pairs
        if ind > 6:
            break
        
        # Convert each input and output to corresponding hex values
        inpString = []
        outString = []
        for hexi in iline.strip().split(" "):
            inpString.append(block_to_byte(hexi)[ind])
        for hexo in oline.strip().split(" "):
            outString.append(block_to_byte(hexo)[ind+1])
        
        # Iterate over all possible pairs of exponents and diagonal values
        for p1, e1 in zip(poss_exp[ind+1], poss_aii[ind+1][ind+1]):
            for p2, e2 in zip(poss_exp[ind], poss_aii[ind][ind]):
                
                # Iterate over all possible values of i
                for i in range(1, 128):
                    flag = True
                    # Iterate over each input and output pair
                    for inp, outp in zip(inpString, outString):
                        # Substitute the pairs ad=nd and iterate over all possible values of i
                        # Form a virtual triangle (aii,aij,ajj)
                        if ord(outp) != Expo(Add(Multiply(Expo(Multiply(Expo(ord(inp), p2), e2), p2), i), Multiply(Expo(Multiply(Expo(ord(inp), p2), i), p1), e1)), p1):
                            flag = False
                            break
                    if flag:
                        # If we find such a value, we can discard other possibilities and finalize the lists
                        poss_exp[ind+1] = [p1]
                        poss_aii[ind+1][ind+1] = [e1]
                        poss_exp[ind] = [p2]
                        poss_aii[ind][ind] = [e2]
                        poss_aii[ind][ind+1] = [i]
                        
# Print the final lists of all possible diagonal values and exponents
print(poss_aii)
print(poss_exp)

[[[84], [124], [], [], [], [], [], []], [[], [70], [29], [], [], [], [], []], [[], [], [43], [25], [], [], [], []], [[], [], [], [12], [119], [], [], []], [[], [], [], [], [112], [101], [], []], [[], [], [], [], [], [11], [95], []], [[], [], [], [], [], [], [27], []], [[], [], [], [], [], [], [], [38, 61, 125]]]
[[18], [118], [41], [74], [91], [53], [26], [17, 41, 69]]


In [76]:
# This function takes in a plaintext string, linear matrix, and exponent matrix as input
def EAEAE(plaintext, lin_mat, exp_mat):

    # Convert the plaintext string into a list of ASCII values
    plaintext = [ord(c) for c in plaintext]

    # Initialize a 2D list to store the intermediate output values
    Output = [[0 for j in range (8)] for i in range(8)]

    # Step 1: Compute the exponential values for each plaintext character
    for ind, elem in enumerate(plaintext):
        Output[0][ind] = Expo(elem, exp_mat[ind])

    # Step 2: Apply the linear transformation to the exponential values obtained in Step 1
    Output[1] = LinearTransformation(lin_mat, Output[0])

    # Step 3: Compute the exponential values for the output of Step 2
    for ind, elem in enumerate(Output[1]):
        Output[2][ind] = Expo(elem, exp_mat[ind])

    # Step 4: Apply the linear transformation to the exponential values obtained in Step 3
    Output[3] = LinearTransformation(lin_mat, Output[2])

    # Step 5: Compute the final exponential values by exponentiating the output of Step 4
    for ind, elem in enumerate(Output[3]):
        Output[4][ind] = Expo(elem, exp_mat[ind])

    # Return the final exponential values as the output of the function
    return Output[4]


In [77]:
for index in range(6):
    #As we have already found element next to diagonal thus skipping two elements every time
    of = index + 2
    
    exp_list = [e[0] for e in poss_exp]
    lin_trans_list = [[0 for i in range(8)] for j in range(8)]
    #We fill all the empty [] elements with 0
    for i in range(8):
        for j in range(8):
            lin_trans_list[i][j] = 0 if len(poss_aii[i][j]) == 0 else poss_aii[i][j][0]
    inp = open("inputs.txt", 'r')
    out = open("outputs.txt", 'r')
    for ind, (iline, oline) in enumerate(zip(inp.readlines(), out.readlines())):
        if ind > (7-of):
            continue
        inpString = [block_to_byte(msg) for msg in iline.strip().split(" ")]
        outString = [block_to_byte(msg) for msg in oline.strip().split(" ")]
        #We iterate over all possible values of ai,j to find which one satisfies EAEAE = Output
        for i in range(1, 128):
            lin_trans_list[ind][ind+of] = i
            flag = True
            for inps, outs in zip(inpString, outString):
                if EAEAE(inps, lin_trans_list, exp_list)[ind+of] != ord(outs[ind+of]):
                    flag = False
                    break
            if flag:
                poss_aii[ind][ind+of] = [i]
    inp.close();
    out.close();
#We fill all the empty [] elements with 0
lin_trans_list = [[0 for i in range(8)] for j in range(8)]
for i in range(8):
    for j in range(8):
        lin_trans_list[i][j] = 0 if len(poss_aii[i][j]) == 0 else poss_aii[i][j][0]

print(lin_trans_list)
print(exp_list)

[[84, 124, 14, 99, 101, 29, 21, 89], [0, 70, 29, 23, 37, 44, 119, 9], [0, 0, 43, 25, 2, 28, 20, 81], [0, 0, 0, 12, 119, 54, 100, 24], [0, 0, 0, 0, 112, 101, 28, 21], [0, 0, 0, 0, 0, 11, 95, 70], [0, 0, 0, 0, 0, 0, 27, 0], [0, 0, 0, 0, 0, 0, 0, 38]]
[18, 118, 41, 74, 91, 53, 26, 17]


In [78]:
#Final Matrices as we found them in above step
LINEAR_MATRIX = [[84, 124, 14, 99, 101, 29, 21, 89],
                 [0, 70, 29, 23, 37, 44, 119, 9], 
                 [0, 0, 43, 25, 2, 28, 20, 81], 
                 [0, 0, 0, 12, 119, 54, 100, 24],
                 [0, 0, 0, 0, 112, 101, 28, 21],
                 [0, 0, 0, 0, 0, 11, 95, 70],
                 [0, 0, 0, 0, 0, 0, 27, 0], 
                 [0, 0, 0, 0, 0, 0, 0, 38]]
EXPONENT_MATRIX = [18, 118, 41, 74, 91, 53, 26, 17]

In [79]:
# Two halves of password
password_1 = "ijghhkirlfhfinlq"
password_2 = "mnllinhiflkofthn"

# We iterate over all possible 128 byte values and perform EAEAE to check for which input the password (half) matches
def DecryptPassword(password):
    passw = block_to_byte(password)
    op = ""
    for ind in range(8):
        for ans in range(128):
            # Create an input block for EAEAE using the current password guess
            inp = op + byte_str(ans)+(16-len(op)-2)*'f'
            # Check if the guess satisfies the EAEAE condition
            if ord(passw[ind]) == EAEAE(block_to_byte(inp), LINEAR_MATRIX, EXPONENT_MATRIX)[ind]:
                op += byte_str(ans)
                break
    return op

# Concatenate the two halves of the decrypted password and print
print(block_to_byte(DecryptPassword(password_1))+block_to_byte(DecryptPassword(password_2)))


rwrpczsncd000000
